In [1]:
import spacy

In [2]:
# ensure to use this on your terminal python -m spacy download en_core_web_sm

nlp = spacy.load("en_core_web_trf")

In [3]:
nlp.pipe_names

['transformer', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']

In [4]:
doc = nlp("Red blouse are good")

for ent in doc.ents:
    print(ent.text, "|", ent.label_, "|", spacy.explain(ent.label_))

In [5]:
from spacy import displacy
displacy.render(doc, style="ent", jupyter=True)

C:\Users\mikos\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\spacy\displacy\__init__.py:213: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  warnings.warn(Warnings.W006)


## Fine tuning Spacy on Custom Dataset

In [ ]:
training_data = []
for example in 